# Multi-Label Defect Prediction using Deep Neural Network

This notebook implements a Deep Neural Network (DNN) model using PyTorch for multi-label defect prediction.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import hamming_loss, classification_report, f1_score, precision_score, recall_score
import joblib
import sys
sys.path.append('..')
from utils.defect_utils import preprocess_data, get_defect_types

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Define the DNN Model

In [ ]:
class DefectPredictionDNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size1=128, hidden_size2=64, dropout_rate=0.3):
        super(DefectPredictionDNN, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size2, output_size),
            nn.Sigmoid()  # Sigmoid for multi-label classification
        )
    
    def forward(self, x):
        return self.model(x)

## Load and Explore Dataset

In [ ]:
# Load the dataset
df = pd.read_csv('../data/dataset.csv')

# Display the first few rows
print("Dataset shape:", df.shape)
df.head()

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values:")
print(missing_values[missing_values > 0] if any(missing_values > 0) else "No missing values")

In [ ]:
# Analyze the defect labels
defect_counts = {}
for defects in df['defects']:
    for defect in defects.split(','):
        defect_counts[defect] = defect_counts.get(defect, 0) + 1

# Convert to DataFrame for visualization
defect_df = pd.DataFrame(list(defect_counts.items()), columns=['Defect', 'Count'])
defect_df = defect_df.sort_values('Count', ascending=False)

# Plot defect distribution
plt.figure(figsize=(12, 6))
sns.barplot(x='Defect', y='Count', data=defect_df)
plt.title('Defect Distribution')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Check for class imbalance
print("Defect distribution:")
for defect, count in defect_counts.items():
    print(f"{defect}: {count} ({count/len(df):.2%})")

## Preprocess Data

In [ ]:
# Preprocess the data
# Extract features (all columns except 'defects')
feature_cols = [col for col in df.columns if col != 'defects']
X = df[feature_cols].values

# Process labels
defects = df['defects'].str.split(',').tolist()

# Convert to multi-hot encoding
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(defects)

# Get the defect class names
defect_classes = mlb.classes_
print(f"Defect classes: {defect_classes}")
print(f"Number of classes: {len(defect_classes)}")

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Feature matrix shape: {X_scaled.shape}")
print(f"Label matrix shape: {y.shape}")

## Split Data

In [ ]:
# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

## Prepare PyTorch Datasets and DataLoaders

In [ ]:
# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)

X_val_tensor = torch.FloatTensor(X_val)
y_val_tensor = torch.FloatTensor(y_val)

X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

## Initialize and Train the DNN Model

In [ ]:
# Initialize the model
input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = DefectPredictionDNN(input_size=input_size, output_size=output_size).to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training parameters
num_epochs = 20
train_losses = []
val_losses = []
val_f1_scores = []

# Training loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)
    
    # Validation phase
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            
            # Convert outputs to binary predictions
            preds = (outputs > 0.5).float()
            
            # Store predictions and labels for metric calculation
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    # Concatenate all batches
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)
    
    # Calculate metrics
    epoch_val_loss = running_loss / len(val_loader.dataset)
    epoch_val_f1 = f1_score(all_labels, all_preds, average='micro')
    
    val_losses.append(epoch_val_loss)
    val_f1_scores.append(epoch_val_f1)
    
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {epoch_train_loss:.4f} - "
          f"Val Loss: {epoch_val_loss:.4f} - "
          f"Val F1: {epoch_val_f1:.4f}")

## Visualize Training Progress

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs+1), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs+1), val_f1_scores, label='Validation F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('Validation F1 Score')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Evaluate Model on Test Set

In [ ]:
# Evaluate model on test set
model.eval()
all_preds = []
all_labels = []
all_outputs = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        # Convert outputs to binary predictions
        preds = (outputs > 0.5).float()
        
        # Store predictions, raw outputs, and labels
        all_preds.append(preds.cpu().numpy())
        all_outputs.append(outputs.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Concatenate all batches
all_preds = np.vstack(all_preds)
all_outputs = np.vstack(all_outputs)
all_labels = np.vstack(all_labels)

# Calculate metrics
hamming = hamming_loss(all_labels, all_preds)
micro_f1 = f1_score(all_labels, all_preds, average='micro')
macro_f1 = f1_score(all_labels, all_preds, average='macro')
micro_precision = precision_score(all_labels, all_preds, average='micro')
macro_precision = precision_score(all_labels, all_preds, average='macro')
micro_recall = recall_score(all_labels, all_preds, average='micro')
macro_recall = recall_score(all_labels, all_preds, average='macro')

# Print metrics
print(f"Test Metrics:")
print(f"Hamming Loss: {hamming:.4f}")
print(f"Micro-F1 Score: {micro_f1:.4f}")
print(f"Macro-F1 Score: {macro_f1:.4f}")
print(f"Micro-Precision: {micro_precision:.4f}")
print(f"Macro-Precision: {macro_precision:.4f}")
print(f"Micro-Recall: {micro_recall:.4f}")
print(f"Macro-Recall: {macro_recall:.4f}")

In [ ]:
# Calculate per-class metrics
class_report = classification_report(all_labels, all_preds, target_names=defect_classes, output_dict=True)
class_metrics = pd.DataFrame(class_report).transpose()
class_metrics = class_metrics.drop('accuracy', errors='ignore')

# Display per-class metrics
print("Per-class metrics:")
display(class_metrics)

# Visualize per-class F1 scores
plt.figure(figsize=(12, 6))
sns.barplot(x=class_metrics.index[:-3], y=class_metrics['f1-score'][:-3])
plt.title('F1 Score per Defect Class')
plt.xticks(rotation=45)
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

## Calculate Precision@k

In [ ]:
# Function to calculate Precision@k
def precision_at_k(y_true, y_score, k):
    """Calculate Precision@k for multi-label classification."""
    # Get the indices of the top k predictions for each sample
    top_k_indices = np.argsort(y_score, axis=1)[:, ::-1][:, :k]
    
    # Create a matrix of predictions with 1s at the top k positions
    y_pred_k = np.zeros_like(y_score)
    for i, indices in enumerate(top_k_indices):
        y_pred_k[i, indices] = 1
    
    # Calculate precision
    precision = 0
    for i in range(len(y_true)):
        if np.sum(y_pred_k[i]) > 0:  # Avoid division by zero
            precision += np.sum(y_true[i] & y_pred_k[i]) / np.sum(y_pred_k[i])
    
    return precision / len(y_true)

# Calculate Precision@k for different values of k
k_values = [1, 2, 3]
for k in k_values:
    p_at_k = precision_at_k(all_labels, all_outputs, k)
    print(f"Precision@{k}: {p_at_k:.4f}")

## Visualize Model Predictions

In [ ]:
# Function to visualize predictions for a sample
def visualize_predictions(sample_idx, y_true, y_pred, y_score, class_names):
    """Visualize predictions for a single sample."""
    true_labels = y_true[sample_idx]
    pred_labels = y_pred[sample_idx]
    pred_scores = y_score[sample_idx]
    
    # Create a DataFrame for visualization
    df = pd.DataFrame({
        'Class': class_names,
        'True': true_labels,
        'Predicted': pred_labels,
        'Score': pred_scores
    })
    
    # Sort by prediction score
    df = df.sort_values('Score', ascending=False)
    
    # Create a color map
    colors = []
    for i in range(len(df)):
        if df.iloc[i]['True'] == 1 and df.iloc[i]['Predicted'] == 1:
            colors.append('green')  # True positive
        elif df.iloc[i]['True'] == 0 and df.iloc[i]['Predicted'] == 1:
            colors.append('red')    # False positive
        elif df.iloc[i]['True'] == 1 and df.iloc[i]['Predicted'] == 0:
            colors.append('orange') # False negative
        else:
            colors.append('gray')   # True negative
    
    # Plot
    plt.figure(figsize=(10, 6))
    bars = plt.barh(df['Class'], df['Score'], color=colors)
    plt.axvline(x=0.5, color='black', linestyle='--', alpha=0.7)
    plt.xlabel('Prediction Score')
    plt.title(f'Defect Predictions for Sample {sample_idx}')
    plt.xlim(0, 1)
    
    # Add a legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='green', label='True Positive'),
        Patch(facecolor='red', label='False Positive'),
        Patch(facecolor='orange', label='False Negative'),
        Patch(facecolor='gray', label='True Negative')
    ]
    plt.legend(handles=legend_elements, loc='lower right')
    
    plt.tight_layout()
    plt.show()
    
    return df

# Visualize predictions for a few samples
for sample_idx in range(3):  # Visualize first 3 samples
    print(f"\nSample {sample_idx}:")
    df = visualize_predictions(sample_idx, all_labels, all_preds, all_outputs, defect_classes)
    display(df)

## Save Model

In [ ]:
# Save model, scaler, and label binarizer
model_path = '../models/dnn_defect.pt'
scaler_path = '../models/dnn_defect_scaler.pkl'
mlb_path = '../models/dnn_defect_mlb.pkl'

torch.save(model.state_dict(), model_path)
joblib.dump(scaler, scaler_path)
joblib.dump(mlb, mlb_path)

print(f"Model saved to {model_path}")
print(f"Scaler saved to {scaler_path}")
print(f"MultiLabelBinarizer saved to {mlb_path}")

# Save model architecture information for later loading
model_info = {
    'input_size': input_size,
    'output_size': output_size,
    'hidden_size1': 128,
    'hidden_size2': 64,
    'dropout_rate': 0.3
}
joblib.dump(model_info, '../models/dnn_defect_info.pkl')
print(f"Model info saved to ../models/dnn_defect_info.pkl")